Split the dataset into train dataset and test dataset in the following manner
	Use train_test_split to split the dataset into train and test dataset with test size equal to 0.3(i.e.30%) and random_state equal to 42. Let other parameters have default values.
	Columns except the last column should be the feature matrix (X_train or X_test)
	Last column will be the label vector.

(Common instructions for Q.2, Q.3 and Q.4)

Train the ridge model on the training data with the following parameters:

alpha = 10
solver = 'saga'
tol = 1e-4
random_state = 42

Enter the value of R2 score on the test dataset.


In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('/content/NPPE1_ModelBuilding3.csv')
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.087472,0.002829,1.768235,0.188396,0.117071,0.148148,0.478165,0.720443,0.967195,0.281563,0.428571,0.959596,1.0,0.0,23.358097
1,0.378379,0.022079,1.115629,0.091974,0.066089,0.851852,0.911759,0.785321,0.885001,0.424648,0.285714,0.868687,0.0,1.0,17.268768
2,0.066901,0.003828,-0.536262,0.221188,0.255671,0.296296,0.228024,0.406472,0.980184,0.274376,0.428571,0.767677,0.0,1.0,27.776974
3,0.140645,0.011132,1.323366,0.422514,0.153103,0.148148,0.410679,0.200319,0.861371,0.305006,0.142857,0.848485,0.0,1.0,16.121960
4,0.144225,0.204918,-0.930790,0.148694,0.177490,0.259259,0.146832,0.111429,0.983448,0.286322,0.285714,0.616162,1.0,0.0,23.129426


In [3]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [4]:
X.shape

(4000, 14)

In [5]:
y.shape

(4000,)

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [7]:
X_train.shape

(2800, 14)

In [8]:
X_test.shape

(1200, 14)

In [9]:
y_train.shape

(2800,)

In [10]:
y_test.shape

(1200,)

Train the ridge model on the training data with the following parameters:

alpha = 10 solver = 'saga' tol = 1e-4 random_state = 42

In [12]:
from sklearn.linear_model import Ridge
model_ridge = Ridge(
    alpha=10,
    solver='saga',
    tol=1e-4,
    random_state=42
)


In [13]:
model_ridge.fit(X_train,y_train)

Ridge(alpha=10, random_state=42, solver='saga')

In [14]:
y_pred = model_ridge.predict(X_test)

In [16]:
from sklearn.metrics import r2_score
r2score = r2_score(y_test, y_pred)
print(r2score)

0.6613547575262211


What is the index of most important feature? Note the index starts from 0. Ignore the intercept for this question.

In [17]:
coefficients = model_ridge.coef_
print(coefficients)

[ -0.4825441    3.74601838  -0.73583331   0.54199933  -9.89014109
   5.80114296  -5.06099736  -9.45015598   4.73124885 -23.51321982
  11.31863371   0.49450664  -0.89196134   0.89196134]


In [18]:
most_important_index = np.argmax(np.abs(coefficients))
print(most_important_index)

9


In [19]:
least_important_index = np.argmin(np.abs(coefficients))
print(least_important_index)

0


Take SGDRegressor(random_state = 42) estimator with GridSearchCV. Hyperparameter tuning to be done over the following parameters:
penalty as ['l1', 'l2']
alpha values as [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
values of tol as [1e-4, 1e-3, 1e-2, 1e-1]
Use cross-validation = 5
Set scoring as neg_mean_absolute_error
Use the best model from above hyper parameter tuning process to answer following questions:

What is the best penalty?

In [5]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV

In [7]:
param_grid = {'penalty':['l1', 'l2'],
              'alpha':[1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
              'tol':[1e-4, 1e-3, 1e-2, 1e-1] }

In [8]:
sgdr_model = SGDRegressor(random_state=42)

In [10]:
grid_searchSGDR = GridSearchCV(
    param_grid=param_grid,
    estimator=sgdr_model,
    scoring='neg_mean_absolute_error',
    cv=5
)

In [11]:
grid_searchSGDR.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=SGDRegressor(random_state=42),
             param_grid={'alpha': [1e-05, 0.0001, 0.001, 0.01, 0.1],
                         'penalty': ['l1', 'l2'],
                         'tol': [0.0001, 0.001, 0.01, 0.1]},
             scoring='neg_mean_absolute_error')

In [12]:
grid_searchSGDR.best_params_

{'alpha': 0.001, 'penalty': 'l2', 'tol': 0.0001}

In [13]:
from sklearn.metrics import mean_absolute_error
y_pred = grid_searchSGDR.predict(X_test)

In [14]:
mae = mean_absolute_error(y_test, y_pred)
print(mae)

3.8131121797994014


Create a pipeline of the PCA() as transformer and Lasso as an estimator.
Use GridSearchCV for tuning the hyperparameters of the created pipeline on training dataset.
	Values of n_components for PCA to be [0.9, 0.95]
	lasso alpha value to be taken as : [10, 1, 0.01, 0.001]
	scoring : neg_mean_absolute_error.
	Use 5 fold cross validation.
	n_jobs = -1 (negative one) [it helps in using all the computational power to run this job]
(Note: Kindly ignore the warning.)

If we fit the pipeline on the training dataset, what will be the R2 score on the test dataset?

In [17]:
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline

In [18]:
pipeline = Pipeline([
    ('pca', PCA()),
    ('lasso', Lasso())
])



In [28]:
param_grid = {'pca__n_components':[0.9, 0.95],
              'lasso__alpha':[10, 1, 0.01, 0.001]}

In [29]:
grid_search_PCAlasso = GridSearchCV(
    param_grid = param_grid,
    estimator=pipeline,
    cv=5,
    n_jobs=-1,
    scoring='neg_mean_absolute_error'
)

In [30]:
grid_search_PCAlasso.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('pca', PCA()), ('lasso', Lasso())]),
             n_jobs=-1,
             param_grid={'lasso__alpha': [10, 1, 0.01, 0.001],
                         'pca__n_components': [0.9, 0.95]},
             scoring='neg_mean_absolute_error')

In [31]:
y_pred=grid_search_PCAlasso.predict(X_test)

In [34]:
from sklearn.metrics import r2_score
r2score = r2_score(y_test, y_pred)
print(r2score)

0.6288625430197575


In [35]:
best_pca = grid_search_PCAlasso.best_estimator_.named_steps['pca']

In [36]:
first_pc_variance = best_pca.explained_variance_[0]

How much variance is explained (Eigen value) by the first principle component?

In [37]:
print("Variance explained by the first principle component is(Eigen value):",first_pc_variance)

Variance explained by the first principle component is(Eigen value): 1.1635075742239045
